In [66]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import ydata_profiling
from time import time

import warnings
warnings.filterwarnings('ignore')

playlist_data = pd.read_csv("../Super_Grp_1_Proj_3/InjuryData/PlayList.csv")
injury_data = pd.read_csv("../Super_Grp_1_Proj_3/InjuryData/InjuryRecord.csv")
#tracking_data = pd.read_csv("../Super_Grp_1_Proj_3/InjuryData/PlayerTrackData.csv",nrows=int(100000)) # due to massive size i've reduce amount of data read 




In [45]:
# pull data the nfl.import_injuries
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
sample_size = 5
injuries = nfl.import_injuries(years)
injuries.sample(sample_size)



,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
2746,2010.0,REG,JAX,12.0,00-0025435,LB,Justin Durant,Justin,Durant,Shoulder,None,Probable,Shoulder,None,Full Participation in Practice,2010-11-26 10:29:19+00:00
3453,2015.0,REG,IND,13.0,00-0026307,LB,Erik Walden,Erik,Walden,Heel,None,Probable,Heel,None,Full Participation in Practice,2015-12-04 10:09:37+00:00
4001,2009.0,REG,ATL,16.0,00-0024372,S,Charlie Peprah,Charlie,Peprah,Hamstring,None,Questionable,Hamstring,None,Did Not Participate In Practice,NaT
3446,2010.0,REG,SF,14.0,00-0011970,K,Joe Nedney,Joe,Nedney,right Knee,None,Out,right Knee,None,Out (Definitely Will Not Play),2010-12-10 12:59:50+00:00
3975,2021.0,REG,NO,14.0,00-0035316,LB,Kaden Elliss,Kaden,Elliss,Hamstring,None,Out,Hamstring,None,Limited Participation in Practice,2021-12-10 20:52:08+00:00


In [18]:
# export data to csv
injuries_data_export.to_csv("Resources/injury_data.csv", index=False, encoding="utf-8")

# Test the CSV Data

In [67]:
playlist_data.head()

,PlayerKey,GameID,PlayKey,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
0,26624,26624-1,26624-1-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,1,QB,QB
1,26624,26624-1,26624-1-2,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,2,QB,QB
2,26624,26624-1,26624-1-3,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,3,QB,QB
3,26624,26624-1,26624-1-4,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,4,QB,QB
4,26624,26624-1,26624-1-5,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,5,QB,QB


In [68]:
injury_data.head()

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1
3,46646,46646-3,46646-3-30,Ankle,Natural,1,0,0,0
4,43532,43532-5,43532-5-69,Ankle,Synthetic,1,1,1,1


In [44]:
# tracking_data.head()

,PlayKey,time,event,x,y,dir,dis,o,s
0,26624-1-1,0.0,huddle_start_offense,87.46,28.93,288.24,0.01,262.33,0.13
1,26624-1-1,0.1,NaN,87.45,28.92,283.91,0.01,261.69,0.12
2,26624-1-1,0.2,NaN,87.44,28.92,280.40,0.01,261.17,0.12
3,26624-1-1,0.3,NaN,87.44,28.92,278.79,0.01,260.66,0.10
4,26624-1-1,0.4,NaN,87.44,28.92,275.44,0.01,260.27,0.09


# Synthetic vs Natural grass field types 

In [69]:
natural_field_grass = injury_data[injury_data.Surface == 'Natural']
turf_field_grass = injury_data[injury_data.Surface == 'Synthetic']

In [70]:
natural_field_grass.head()

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
3,46646,46646-3,46646-3-30,Ankle,Natural,1,0,0,0
5,41145,41145-2,41145-2-60,Knee,Natural,1,0,0,0
7,44860,44860-5,44860-5-52,Knee,Natural,1,1,0,0
11,36621,36621-13,36621-13-58,Foot,Natural,1,1,1,1


In [71]:
turf_field_grass.head()

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1
4,43532,43532-5,43532-5-69,Ankle,Synthetic,1,1,1,1
6,46014,46014-10,46014-10-22,Ankle,Synthetic,1,1,1,1
8,44806,44806-7,44806-7-61,Knee,Synthetic,1,0,0,0


# Cleaning Up Data

In [72]:
# Merge both natural_field_grass and turf_field_grass with the PlayList DataTable 
# Then merge the PlayList and Injury tables together

nat_grass_type = pd.merge(natural_field_grass, playlist_data, on=['PlayerKey', 'GameID', 'PlayKey'], how='left')
turf_grass_type = pd.merge(turf_field_grass, playlist_data, on=['PlayerKey', 'GameID', 'PlayKey'], how='left')
injury_playlist_merge = pd.merge(injury_data, playlist_data, on=['PlayerKey', 'GameID', 'PlayKey'], how='left')

#NOTE: Will be using a pd.get_dummies to convert categorical variable into dummy/indicator variables. 
# I.E. changing from 'BodyPart' category field column to 'part' and 'RosterPosition' to 'pos'

inj_play_concat = pd.concat([injury_playlist_merge, 
                            pd.get_dummies(injury_playlist_merge['BodyPart'], prefix='part'),
                             pd.get_dummies(injury_playlist_merge['RosterPosition'],prefix='pos')], axis=1)


In [73]:
injury_playlist_merge.sample(sample_size)

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
13,43505,43505-2,43505-2-49,Foot,Natural,1,1,1,1,Offensive Lineman,11.0,2.0,Outdoor,Natural,68.0,Cloudy,Pass,49.0,C,OL
31,31070,31070-3,31070-3-7,Knee,Natural,1,1,1,1,Running Back,15.0,3.0,Outdoor,Natural,89.0,Sunny,Rush,7.0,RB,RB
100,44423,44423-13,NaN,Knee,Synthetic,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,43540,43540-7,43540-7-2,Ankle,Natural,1,1,1,1,Linebacker,44.0,7.0,Outdoor,Natural,70.0,Cloudy,Pass,2.0,MLB,LB
30,44900,44900-7,44900-7-10,Knee,Natural,1,1,0,0,Defensive Lineman,43.0,7.0,Outdoor,Natural,70.0,"Cloudy with periods of rain, thunder possible....",Kickoff,10.0,DE,DL


In [74]:
inj_play_concat.sample(sample_size)


,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42,RosterPosition,...,part_Knee,part_Toes,pos_Cornerback,pos_Defensive Lineman,pos_Linebacker,pos_Offensive Lineman,pos_Running Back,pos_Safety,pos_Tight End,pos_Wide Receiver
94,47334,47334-8,47334-8-1,Knee,Synthetic,1,1,0,0,Safety,...,True,False,False,False,False,False,False,True,False,False
85,33337,33337-8,33337-8-15,Foot,Natural,1,1,1,0,Tight End,...,False,False,False,False,False,False,False,False,True,False
95,46134,46134-18,NaN,Heel,Natural,1,1,0,0,NaN,...,False,False,False,False,False,False,False,False,False,False
77,44449,44449-28,44449-28-35,Ankle,Synthetic,1,0,0,0,Wide Receiver,...,False,False,False,False,False,False,False,False,False,True
72,36559,36559-12,36559-12-65,Knee,Natural,1,1,0,0,Cornerback,...,True,False,True,False,False,False,False,False,False,False
